In [4]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
import seaborn as sns
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import pymysql

import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages

import scipy.stats as ss

# ignore warnings
import warnings
warnings.filterwarnings('ignore')#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
import seaborn as sns
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import pymysql

import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages

import scipy.stats as ss

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Import relevant modules and define connection
from sshtunnel import SSHTunnelForwarder
import pymysql
import pandas as pd

server = SSHTunnelForwarder(
ssh_address=('ipa-008.ucd.ie', 22),
ssh_username="wingki",
ssh_password="6!#5/6_Dublinbus",
remote_bind_address=("localhost", 3306))

server.start()

con = pymysql.connect(user='root',passwd='Dublinbus_6!#5/6',db='dublinbus',host='localhost',port=server.local_bind_port)


In [6]:
#read in data
routeData = pd.read_sql_query("SELECT * FROM dublinbus.route_1;", con)

In [7]:
routeData.head(25)

,DAYOFSERVICE,TRIPID,LINEID,PROGRNUMBER,STOPPOINTID,DIRECTION,ACTUALTIME_DEP,ACTUALTIME_ARR
0,07-FEB-18 00:00:00,6253554,1,1,381,2,63629,63629
1,07-FEB-18 00:00:00,6253554,1,2,382,2,63690,63675
2,07-FEB-18 00:00:00,6253554,1,3,4451,2,63739,63715
3,07-FEB-18 00:00:00,6253554,1,4,383,2,63788,63774
4,07-FEB-18 00:00:00,6253554,1,5,384,2,63824,63824
5,07-FEB-18 00:00:00,6253554,1,6,385,2,63897,63869
6,07-FEB-18 00:00:00,6253554,1,8,388,2,64084,64084
7,07-FEB-18 00:00:00,6253554,1,9,389,2,64097,64097
8,07-FEB-18 00:00:00,6253554,1,10,393,2,64283,64283
9,07-FEB-18 00:00:00,6253554,1,11,371,2,64344,64314


### Clean Data

In [8]:
routeData['hour'] = pd.to_datetime(routeData['ACTUALTIME_DEP'], unit='s').dt.hour

In [9]:
routeData['DAYOFSERVICE'] = pd.to_datetime(routeData.DAYOFSERVICE)

In [10]:
# Create new column dayofweek based on the DAYOFSERVICE
# if it is a weekday, value=0; if it is a weekend, value=1
conditions = [
    (routeData['DAYOFSERVICE'].dt.dayofweek == 5),(routeData['DAYOFSERVICE'].dt.dayofweek == 6)
    ]
choices = [1,1]
routeData['dayofweek'] = np.select(conditions, choices, default=0)# Create new column dayofweek based on the DAYOFSERVICE


In [11]:
routeData['DIRECTION'] = routeData['DIRECTION'].astype('int64')

In [12]:
numeric_columns = routeData.select_dtypes(['int64']).columns
columns = list(routeData)

### Get Weather Data

In [13]:
weatherData = pd.read_csv('weather-data-cleaned.csv', keep_default_na=True, delimiter=',')

In [14]:
weatherData.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [15]:
weatherData['hour'] = pd.DatetimeIndex(weatherData['datetime']).hour

In [16]:
weatherData['datetime'] = pd.to_datetime(weatherData['datetime']).dt.date

In [17]:
weatherData.rename(columns = {'datetime':'DAYOFSERVICE'}, inplace = True)

In [18]:
weatherData['DAYOFSERVICE'] = pd.to_datetime(weatherData.DAYOFSERVICE)

In [19]:
#Combine Weather and route data
df=pd.merge(routeData, weatherData, how='inner', left_on=['DAYOFSERVICE', 'hour'], right_on=['DAYOFSERVICE','hour'])

In [20]:
#Get holiday data
holidayData = pd.read_csv('ireland-holidays-2018-list-classic-en-ie.csv', keep_default_na=True, delimiter=',')

In [29]:
#drop all rows that have any NaN values
holidayData.dropna(inplace=True) 

In [22]:
holidayData['date'] = pd.to_datetime(holidayData.date)

In [23]:
#Create a daily datetime index using the start and end times from a period with yearly frequency
#reference: https://stackoverflow.com/questions/20602947/append-column-to-pandas-dataframe
year = '2018'
df1 = pd.DataFrame({
    'Date': pd.date_range(
        start = pd.Timestamp(year),                        
        end = pd.Timestamp(year) + pd.offsets.YearEnd(0),  
        freq = 'D'
    )
})

In [24]:
# join the Date to holidayData
holidayData=df1.join(holidayData)

In [25]:
# create a new column "holiday in holidayData" and set default cell value to be 0, indicating not a holiday
# set the dates of summer holiday to 2
start_date='2018-06-21'
end_date='2018-09-23'
conditions = [
    (holidayData['Date'] >= start_date) & (holidayData['Date'] <= end_date)
    ]
choices = [2]
holidayData['holiday'] = np.select(conditions, choices,default=0)

In [26]:
# flip those dates with bank holiday to 1 in "holiday" column
holidayData.loc[holidayData['Date']=='2018-01-01', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-03-11', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-03-17', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-03-19', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-03-30', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-04-01', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-04-02', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-05-07', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-06-04', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-08-06', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-10-29', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-12-25', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-12-26', 'holiday'] = 1
holidayData.loc[holidayData['Date']=='2018-12-27', 'holiday'] = 1

In [27]:
#check again if flip based on the right bank holidays
holidayData[holidayData['holiday']==1]

,Date,date,nameOfHoliday,holiday
0,2018-01-01,2018-01-01,New Year's Day,1
69,2018-03-11,NaT,NaN,1
75,2018-03-17,NaT,NaN,1
77,2018-03-19,NaT,NaN,1
88,2018-03-30,NaT,NaN,1
90,2018-04-01,NaT,NaN,1
91,2018-04-02,NaT,NaN,1
126,2018-05-07,NaT,NaN,1
154,2018-06-04,NaT,NaN,1
217,2018-08-06,NaT,NaN,1


In [28]:
#drop nameOfHoliday and date that won't be used
holidayData.drop(['nameOfHoliday'], axis = 1, inplace=True) 
holidayData.drop(['date'], axis = 1, inplace=True) 

In [30]:
#rename the date column to make it more convenient for future joining
holidayData.rename(columns = {'Date':'DAYOFSERVICE'}, inplace = True)

In [31]:
df=pd.merge(df, holidayData, how='inner', left_on=['DAYOFSERVICE'], right_on=['DAYOFSERVICE'])

In [37]:
df

,DAYOFSERVICE,TRIPID,LINEID,PROGRNUMBER,STOPPOINTID,DIRECTION,ACTUALTIME_DEP,ACTUALTIME_ARR,hour,dayofweek,...,temp,pressure,humidity,wind_speed,wind_dir,sun,visibility,cloud_height,cloud_cover,holiday
0,2018-02-07,6253554,1,1,381,2,63629,63629,17,0,...,2.9,6.7,6,9,210,0.0,25000,40,7,0
1,2018-02-07,6253554,1,2,382,2,63690,63675,17,0,...,2.9,6.7,6,9,210,0.0,25000,40,7,0
2,2018-02-07,6253554,1,3,4451,2,63739,63715,17,0,...,2.9,6.7,6,9,210,0.0,25000,40,7,0
3,2018-02-07,6253554,1,4,383,2,63788,63774,17,0,...,2.9,6.7,6,9,210,0.0,25000,40,7,0
4,2018-02-07,6253554,1,5,384,2,63824,63824,17,0,...,2.9,6.7,6,9,210,0.0,25000,40,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336955,2018-01-07,6105447,1,35,373,1,85448,85448,23,1,...,1.9,4.7,4,10,100,0.0,30000,999,3,0
1336956,2018-01-07,6105447,1,13,44,1,83332,83320,23,1,...,1.9,4.7,4,10,100,0.0,30000,999,3,0
1336957,2018-01-07,6105447,1,34,372,1,85428,85428,23,1,...,1.9,4.7,4,10,100,0.0,30000,999,3,0
1336958,2018-01-07,6105447,1,1,226,1,82827,82827,23,1,...,1.9,4.7,4,10,100,0.0,30000,999,3,0


In [33]:
df.to_csv('d_mod.csv')

In [38]:
df['trip_dep_time'] = ""

In [ ]:
dfbver